In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import decimal
import ast
import os
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import pickle
import datetime

In [ ]:
path=''

In [ ]:
! pip install sentence_transformers

In [ ]:
import os 
import ast
import sentence_transformers

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity ### Cosine Similary
from scipy import sparse ### Sparse Matrix

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [ ]:

raw_interactions=pd.read_csv('path/RAW_interactions.csv')
raw_recipes=pd.read_csv('path/RAW_recipes.csv')

# 전처리

## 레시피 작성자가 작성한 리뷰 제거

In [ ]:
raw_recipes=raw_recipes.rename(columns={"id": "recipe_id"})

In [ ]:
# 본인이 만든 레시피 삭제(raw_interactions_1)
raw_interactions_1=pd.merge(raw_interactions,raw_recipes[['recipe_id','contributor_id']] , on='recipe_id')

In [ ]:
raw_interactions_1.drop(raw_interactions_1[raw_interactions_1.contributor_id==raw_interactions_1.user_id][raw_interactions.columns].index.tolist(), inplace=True)

In [ ]:
raw_interactions_1.drop(['contributor_id'], axis=1, inplace=True)

### name 없는 row 삭제


In [ ]:
raw_recipes['name'][raw_recipes['name'].isna()]

In [ ]:
raw_recipes.drop(721, inplace=True)

### minutes에서 0분 삭제 1일 초과 삭제

In [ ]:
# 0분 삭제
raw_recipes[raw_recipes.minutes==0].index
raw_recipes.drop(raw_recipes[raw_recipes.minutes==0].index, inplace=True)

In [ ]:
raw_recipes[raw_recipes.minutes>1440].index
raw_recipes.drop(raw_recipes[raw_recipes.minutes>1440].index, inplace=True)

In [ ]:
raw_recipes.head(2)

### interaction, recipe 결합

In [ ]:
# 리뷰 2건 이상인 레시피만 필터링

### Print for the poc purposes we will restrict
g = {'rating' : ['mean'],'user_id' : ['nunique']}
int_summary = raw_interactions_1.groupby(['recipe_id']).agg(g).reset_index()

### Its gives a muti index output convert it to single index by cobining bothe level
ind = pd.Index([e[0] + "_" +e[1] for e in int_summary.columns.tolist()])

### Assign the column names 
int_summary.columns = ind
int_summary.columns = ['recipe_id', 'rating_mean', 'user_id_nunique']

### We will keep only those recipes in considerstaion which have been reviewed by more than 2 people
int_summary_2 = int_summary[ (int_summary['user_id_nunique'] > 2)]

In [ ]:
filter_recipe = pd.merge(raw_recipes,int_summary_2,right_on = ['recipe_id'],left_on = ['recipe_id'],how = 'inner')

## steps별 코사인 유사도

In [ ]:
final_review=pd.merge(filter_recipe, raw_interactions_1 , on='recipe_id')[['user_id', 'recipe_id', 'rating']]

In [ ]:
filter_recipe['dish_recipe'] = filter_recipe['steps'].apply(lambda x : " ".join(ast.literal_eval(x)))

In [ ]:
encodings_recipe= model.encode(filter_recipe['dish_recipe'])
pickle.dump(encodings_recipe,open("recipe_embedding.pickle",'wb'))

In [ ]:
with open("path/recipe_embedding.pickle", "wb") as MyFile:
    pickle.dump(encodings_recipe, MyFile)

In [ ]:
#### Load the pickle files of encoding and create a dataframe out of it
encodings_recipe_df = pickle.load(open("recipe_embedding.pickle",'rb'))
print ("Encoding are loaded")
data_encoding = pd.DataFrame(encodings_recipe_df)

In [ ]:
len(encodings_recipe_df)

In [ ]:
filter_recipe['tag_recipe']=filter_recipe['tags'].apply(lambda x : " ".join(ast.literal_eval(x)))

In [ ]:
encodings_tag= model.encode(filter_recipe['tag_recipe'])
pickle.dump(encodings_tag,open("tag_embedding.pickle",'wb'))

In [ ]:
with open("path/tag_embedding.pickle", "wb") as MyFile:
    pickle.dump(encodings_tag, MyFile)

In [ ]:
#### Load the pickle files of encoding and create a dataframe out of it
encodings_tag_df = pickle.load(open("tag_embedding.pickle",'rb'))
print ("Encoding are loaded")
tag_encoding = pd.DataFrame(encodings_tag_df)

In [ ]:
data_encoding

In [ ]:
recipe_id_dict = {}
for j,i in enumerate(filter_recipe['recipe_id']):
    recipe_id_dict[j] = i
print ("Dictionary is created :")

In [ ]:
len(recipe_id_dict.keys())

In [ ]:
data_encoding['recipe_id']=data_encoding.index.map(recipe_id_dict)
id_index_17=final_review.groupby(['recipe_id']).count()[final_review.groupby(['recipe_id']).count()['user_id']>=17].index
data_encoding_17=(data_encoding.set_index('recipe_id')).loc[(data_encoding.set_index('recipe_id')).index.intersection(id_index_17),:]
encodings_recipe_df_17=np.array(data_encoding_17)

In [ ]:
tag_encoding['recipe_id']=tag_encoding.index.map(recipe_id_dict)
id_index_17=final_review.groupby(['recipe_id']).count()[final_review.groupby(['recipe_id']).count()['user_id']>=17].index
tag_encoding_17=(tag_encoding.set_index('recipe_id')).loc[(tag_encoding.set_index('recipe_id')).index.intersection(id_index_17),:]
encodings_tag_df_17=np.array(tag_encoding_17)

In [ ]:
encoding_sparse_recipe = sparse.csr_matrix(encodings_recipe_df)
# encodings_recipe_df_17

In [ ]:
encoding_sparse_tag = sparse.csr_matrix(encodings_tag_df)
# encodings_tag_df_17

In [ ]:
import datetime
time = datetime.datetime.now()
print (time)
similarities_sparse_recipe = cosine_similarity(encoding_sparse_recipe)
# print('pairwise sparse output:\n {}\n'.format(similarities_sparse))
print ("Time taken is :",)
print (datetime.datetime.now()-time)

In [ ]:
time = datetime.datetime.now()
print (time)
similarities_sparse_tag = cosine_similarity(encoding_sparse_tag)
# print('pairwise sparse output:\n {}\n'.format(similarities_sparse))
print ("Time taken is :",)
print (datetime.datetime.now()-time)

In [ ]:
df1 = pd.DataFrame(similarities_sparse_recipe)
df1.index=data_encoding_17.index
df1.columns=data_encoding_17.index

In [ ]:
### Convert the pickle file to datafarme and dump
df1 = pd.DataFrame(similarities_sparse_recipe)
df1.index=data_encoding_17.index
df1.columns=data_encoding_17.index
pickle.dump(df1,open('similarities_sparse_recipe.pickle','wb'))

In [ ]:
with open("path/similarities_sparse_recipe.pickle", "wb") as MyFile:
    pickle.dump(df1, MyFile)

In [ ]:
### Convert the pickle file to datafarme and dump
df2 = pd.DataFrame(similarities_sparse_tag)
df2.index=data_encoding_17.index
df2.columns=data_encoding_17.index
pickle.dump(df2,open('similarities_sparse_tag.pickle','wb'))

In [ ]:
with open("path/similarities_sparse_tag.pickle", "wb") as MyFile:
    pickle.dump(df2, MyFile)